In [73]:
from statsbombpy import sb
import numpy as np
import pandas as pd

In [74]:
matches = sb.matches(competition_id=11, season_id=90)

/home/mustafa/.local/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [76]:
# match_id = 3773386
# events = sb.events(match_id=match_id)

In [77]:
all_finished_passes = []

print(f"Processing {len(match_ids)} matches...")

# Loop through every match
for match in match_ids:
    try:
        # 1. DOWNLOAD the full event data for the match
        events = sb.events(match_id=match)
        events['team_name'] = events['team']
        events['match_id'] = match
        
        # 2. SORT chronologically (Crucial for history features like 'previous action')
        events = events.sort_values(['period', 'timestamp', 'index'])
        
        # --- FEATURE ENGINEERING (Do this BEFORE filtering for passes) ---
        
        # Extract Coordinates
        events[['x', 'y']] = events['location'].apply(pd.Series)
        events[['end_x', 'end_y']] = events['pass_end_location'].apply(pd.Series)
        
        # Geometric Features
        events['pass_distance'] = np.sqrt((events['end_x'] - events['x'])**2 + (events['end_y'] - events['y'])**2)
        events['pass_angle'] = np.arctan2(events['end_y'] - events['y'], events['end_x'] - events['x'])
        events['mirrored_y'] = abs(events['y'] - 40)
        
        # Simple Features (Handle NaNs)
        events['is_cross'] = (events['pass_cross'] == True).astype(int)
        events['body_part_name'] = events['pass_body_part']
        events['play_pattern_name'] = events['play_pattern']
        events['pass_height_name'] = events['pass_height']
        
        # History Features (Requires the full event stream)
        events['is_pass'] = (events['type'] == 'Pass').astype(int)
        
        # Pass Count in Possession
        events['pass_count_in_possession'] = events.groupby(['match_id', 'possession'])['is_pass'].cumsum() - 1
        
        # Previous Action Types
        events['prev_type'] = events['type'].shift(1)
        events['prev_team'] = events['team'].shift(1)
        events['prev_action_is_teammate'] = (events['team'] == events['prev_team']).astype(int)
        
        # Previous Carry Distance
        # (We calculate distance only if the previous event was a 'Carry')
        # We need to extract the previous location safely
        prev_loc = events['location'].shift(1)
        curr_loc = events['location']
        
        # Calculate distance between current event start and previous event start
        # Note: For a carry, the 'location' is the start, and 'carry_end_location' is the end.
        # But logically, if the previous event was a carry, the player is NOW at the carry end location.
        # StatsBomb simplified: The distance covered is simply the carry length.
        
        # An easier way for 'prev_carry_distance':
        # If prev_type is Carry, calculate distance between its start and end.
        # However, calculating it row-by-row is slow. A vectorized approach:
        events['carry_length'] = np.sqrt(
             (events['carry_end_location'].apply(lambda x: x[0] if isinstance(x, list) else np.nan) - 
              events['location'].apply(lambda x: x[0] if isinstance(x, list) else np.nan))**2 + 
             (events['carry_end_location'].apply(lambda x: x[1] if isinstance(x, list) else np.nan) - 
              events['location'].apply(lambda x: x[1] if isinstance(x, list) else np.nan))**2
        )
        events['prev_carry_distance'] = events['carry_length'].shift(1).fillna(0)
        
        # Vertical Velocity
        events['time_seconds'] = events['minute']*60 + events['second']
        events['vertical_velocity'] = (events['x'] - events['x'].shift(3)) / (events['time_seconds'] - events['time_seconds'].shift(3))

        # --- FILTERING ---
        
        # Now we can throw away the non-pass rows
        passes_only = events[events['type'] == 'Pass'].copy()
        
        # Add Success Target
        passes_only['success'] = passes_only['pass_outcome'].isnull().astype(int)
        
        # Select final columns
        features = [
            'player','team_name', 'x', 'mirrored_y', 'pass_angle', 'pass_distance',
            'body_part_name', 'is_cross', 'play_pattern_name', 'pass_height_name',
            'pass_count_in_possession', 'prev_carry_distance', 'vertical_velocity',
            'prev_action_is_teammate', 'prev_type', 'success'
        ]
        
        # Keep only existing columns (avoid errors if a match has no crosses for example)
        # passes_only = passes_only[features] 
        # Safer way:
        passes_only = passes_only.reindex(columns=features)
        
        all_finished_passes.append(passes_only)
        
    except Exception as e:
        print(f"Error in match {match}: {e}")

# 3. COMBINE EVERYTHING
df = pd.concat(all_finished_passes, ignore_index=True)



/home/mustafa/.local/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


Processing 35 matches...


/tmp/ipykernel_12884/3327473043.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  events['pass_count_in_possession'] = events.groupby(['match_id', 'possession'])['is_pass'].cumsum() - 1
/tmp/ipykernel_12884/3327473043.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  events['prev_type'] = events['type'].shift(1)
/tmp/ipykernel_12884/3327473043.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns

In [78]:
# Create your final training set
training_data = events[events['type'] == 'Pass'].copy()

# Select only the columns you listed
features = [
    'player','team_name','x', 'mirrored_y', 'pass_angle', 'pass_distance',
    'body_part_name', 'is_cross', 'play_pattern_name', 'pass_height_name',
    'pass_count_in_possession', 'prev_carry_distance', 'vertical_velocity',
    'prev_action_is_teammate', 'prev_type', 'success'
]


df.head()

,player,team_name,x,mirrored_y,pass_angle,pass_distance,body_part_name,is_cross,play_pattern_name,pass_height_name,pass_count_in_possession,prev_carry_distance,vertical_velocity,prev_action_is_teammate,prev_type,success
0,Antoine Griezmann,Barcelona,61.0,0.1,2.692596,12.210242,Left Foot,0,From Kick Off,Ground Pass,0,0.000000,NaN,0,Half Start,1
1,Sergio Busquets i Burgos,Barcelona,49.4,4.9,-1.720006,15.471910,Right Foot,0,From Kick Off,Ground Pass,1,0.000000,NaN,1,Ball Receipt*,1
2,Frenkie de Jong,Barcelona,48.1,10.4,2.111216,24.489998,Right Foot,0,From Kick Off,Ground Pass,2,1.000000,-0.325,1,Carry,1
3,Gerard Piqué Bernabéu,Barcelona,44.3,10.2,-1.730403,20.763911,Right Foot,0,From Kick Off,Ground Pass,3,8.809086,-0.760,1,Carry,1
4,Clément Lenglet,Barcelona,49.0,11.3,0.499347,10.023971,Left Foot,0,From Kick Off,Ground Pass,4,8.062258,1.175,1,Carry,1


In [79]:
df = df.fillna(value='0')

In [80]:
df.shape

(40337, 16)

In [81]:
df.to_csv("passing_data.csv")